In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# copying image files from drive to colab

!rm -rf /content/test # checking if the exist with same folder and deleting it
!cp -r /content/gdrive/Othercomputers/BFMC/Bfmc_ahduni/data   /content/data

## Creating DataSet

In [9]:
# The image_dataset_from_directory creates dataset from directory
# Creating training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/content/data/training_data',
    labels='inferred',
    class_names=['bicycle', 'goods_vehicles', 'hump', 'no_entry', 'no_left', 'no_overtaking', 'no_parking', 'no_right', 'no_stopping', 'parking', 'priority_to', 'speed_limit', 'stop', 'yield'],
    label_mode='categorical',
    batch_size=64,
    image_size=(224, 224))

# Now, creating testing dataset
validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/content/data/testing_data',
    labels='inferred',
    class_names=['bicycle', 'goods_vehicles', 'hump', 'no_entry', 'no_left', 'no_overtaking', 'no_parking', 'no_right', 'no_stopping', 'parking', 'priority_to', 'speed_limit', 'stop', 'yield'],
    label_mode='categorical',
    batch_size=64,
    image_size=(224, 224))

Found 72918 files belonging to 14 classes.
Found 33340 files belonging to 14 classes.


## Creating the Model

In [10]:
# creating EfficientNetB0 model
model = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights= None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=14,
    classifier_activation="softmax"
)

In [11]:
# Adding optimizer and loss funtion to the model
# Here optimizer is adam and loss function is mean square root
model.compile(optimizer="adam", loss="mse")


In [12]:
model.fit(train_ds, epochs=10, validation_data=validation_ds)

Epoch 1/10
1140/1140 [==============================] - 1382s 1s/step - loss: 0.0191 - val_loss: 0.0270
Epoch 2/10
1140/1140 [==============================] - 1364s 1s/step - loss: 0.0084 - val_loss: 0.0441
Epoch 3/10
1140/1140 [==============================] - 1364s 1s/step - loss: 0.0046 - val_loss: 0.0189
Epoch 4/10
1140/1140 [==============================] - 1364s 1s/step - loss: 0.0022 - val_loss: 0.0161
Epoch 5/10
1140/1140 [==============================] - 1363s 1s/step - loss: 0.0015 - val_loss: 0.0225
Epoch 6/10
1140/1140 [==============================] - 1366s 1s/step - loss: 0.0014 - val_loss: 0.0235
Epoch 7/10
1140/1140 [==============================] - 1362s 1s/step - loss: 0.0011 - val_loss: 0.0231
Epoch 8/10
1140/1140 [==============================] - 1362s 1s/step - loss: 0.0011 - val_loss: 0.0153
Epoch 9/10
1140/1140 [==============================] - 1362s 1s/step - loss: 0.0011 - val_loss: 0.0178
Epoch 10/10
1140/1140 [==============================] - 1363s 1

In [29]:
model_acc = model.evaluate(validation_ds)

521/521 [==============================] - 131s 251ms/step - loss: 0.0155


In [21]:
# creating custom function for decoding predictions
def decode_prediction(preds):
    class_names=['bicycle', 'goods_vehicles', 'hump', 'no_entry', 'no_left', 'no_overtaking', 'no_parking', 'no_right', 'no_stopping', 'parking', 'priority_to', 'speed_limit', 'stop', 'yield']
    last = 0
    for i in range(14):
      last = max(last, preds[0][i])
    k = 0
    for j in preds[0]:
      if(j == last):
        print("predicted output is :", class_names[k])
      k = k + 1


In [26]:
img_path = '/content/data/testing_data/no_entry/01_12_07_02_0039.bmp'
# img_path = '/content/data/testing_data/parking/01_14_07_01_0340.bmp'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = tf.keras.applications.efficientnet.preprocess_input(x)

preds = model.predict(x)
decode_prediction(preds)


predicted output is : no_entry


## Training for CategoricalCrossentropy loss function

In [40]:
# creating EfficientNetB0 model
model2 = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights= None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=14,
    classifier_activation="softmax"
)

In [42]:
# Adding optimizer and loss funtion to the model
# Here optimizer is adam and loss function is mean square root
model2.compile(optimizer="adam", loss="CategoricalCrossentropy", metrics = ["categorical_accuracy" ])

In [ ]:
model2.fit(train_ds, epochs=10, validation_data=validation_ds)

Epoch 1/10
1140/1140 [==============================] - 1383s 1s/step - loss: 0.6346 - categorical_accuracy: 0.8302 - val_loss: 0.8727 - val_categorical_accuracy: 0.8184
Epoch 2/10
1140/1140 [==============================] - 1381s 1s/step - loss: 0.0822 - categorical_accuracy: 0.9737 - val_loss: 1.2039 - val_categorical_accuracy: 0.7972
Epoch 3/10
1140/1140 [==============================] - 1381s 1s/step - loss: 0.0535 - categorical_accuracy: 0.9835 - val_loss: 1.3579 - val_categorical_accuracy: 0.7581
Epoch 4/10
 637/1140 [===============>..............] - ETA: 9:09 - loss: 0.0422 - categorical_accuracy: 0.9868

In [1]:
img_path = '/content/data/testing_data/no_entry/01_12_07_02_0039.bmp'
# img_path = '/content/data/testing_data/parking/01_14_07_01_0340.bmp'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = tf.keras.applications.efficientnet.preprocess_input(x)

preds2 = model2.predict(x)
decode_prediction(preds2)


NameError: ignored